[link text](https://)

# PTSD with Transfer Learning




In [0]:
from fastai.text import * 


# Getting the Data

In [0]:
import pandas as pd
raw_data = pd.read_csv("./PTSD_data - PTSD_data.csv",  skiprows=3)
raw_annotations = pd.read_csv("./PTSD_data - Annotations.csv", skiprows=1)

In [15]:
raw_data.head()



,transcript_id,character,text,Keywords / Significant sentences,A1,B1,B2,B3,B4,B5,C1,C2,D1,D2,D3,D4,D5,D6,D7,E1,E2,E3,E4,E5,E6,F1,G1
0,PTSD_file_1,CLIENT,Remind me never to go to a work meeting with a...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,PTSD_file_1,THERAPIST,Those darn women.,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,PTSD_file_1,CLIENT,Damn women. Our boss is she's just a shut the ...,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,PTSD_file_1,THERAPIST,It's not that comfortable.,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,PTSD_file_1,CLIENT,What's that?,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
raw_annotations.head()

,Session_id,Scorer,A,B,C,D,E,G
0,PTSD_file_1,Alexis,1.0,0.0,1.0,1.0,0.0,0.0
1,PTSD_file_2,Alexis,1.0,1.0,1.0,1.0,1.0,0.0
2,PTSD_file_3,Alexis,0.0,0.0,0.0,0.0,0.0,0.0
3,PTSD_file_4,Alexis,0.0,1.0,1.0,1.0,1.0,0.0
4,PTSD_file_5,Alexis,1.0,0.0,0.0,0.0,0.0,0.0


In [0]:
raw_text_combined = raw_data.groupby("transcript_id")["text"].agg(lambda col: ' '.join(col))


In [0]:
text_transcript_combined = raw_text_combined.to_frame()

In [0]:
text_transcript_combined.reset_index(level=0, inplace=True)

In [32]:
text_transcript_combined.head()

,transcript_id,text
0,PTSD_file_1,Remind me never to go to a work meeting with a...
1,PTSD_file_10,"The arch. How was time? Good, good, thanks. Y..."
2,PTSD_file_11,(yawning) Hi. Hi. I don’t know why I can’t ge...
3,PTSD_file_12,Hi. Come on in. Morning. Morning. Is it less c...
4,PTSD_file_13,(inaudible response) Thanks for not having (i...


In [0]:
raw_annotations["label"] = (raw_annotations["A"]+raw_annotations["B"]+raw_annotations["C"]+raw_annotations["D"]+raw_annotations["E"]+raw_annotations["G"])/6

In [47]:
raw_annotations.head()

,Session_id,Scorer,A,B,C,D,E,G,total,label
0,PTSD_file_1,Alexis,1.0,0.0,1.0,1.0,0.0,0.0,0.500000,0.500000
1,PTSD_file_2,Alexis,1.0,1.0,1.0,1.0,1.0,0.0,0.833333,0.833333
2,PTSD_file_3,Alexis,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000
3,PTSD_file_4,Alexis,0.0,1.0,1.0,1.0,1.0,0.0,0.666667,0.666667
4,PTSD_file_5,Alexis,1.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.166667


In [0]:
final_annotations = raw_annotations.round({"label":0})[["Session_id","label"]]

In [50]:
final_annotations.head()

,Session_id,label
0,PTSD_file_1,0.0
1,PTSD_file_2,1.0
2,PTSD_file_3,0.0
3,PTSD_file_4,1.0
4,PTSD_file_5,0.0


In [0]:
#final_annotations = final_annotations.rename(columns = {"Session_id": "transcript_id"}) 
final_annotations.label = final_annotations.label.fillna(0)
final_annotations.label = final_annotations.label.astype(int)

In [54]:
final_annotations.head()

,transcript_id,label
0,PTSD_file_1,0
1,PTSD_file_2,1
2,PTSD_file_3,0
3,PTSD_file_4,1
4,PTSD_file_5,0


In [0]:
final_dataset = pd.merge(text_transcript_combined, final_annotations, on='transcript_id')


In [59]:
final_dataset.head()

,transcript_id,text,label
0,PTSD_file_1,Remind me never to go to a work meeting with a...,0
1,PTSD_file_10,"The arch. How was time? Good, good, thanks. Y...",1
2,PTSD_file_11,(yawning) Hi. Hi. I don’t know why I can’t ge...,1
3,PTSD_file_12,Hi. Come on in. Morning. Morning. Is it less c...,0
4,PTSD_file_13,(inaudible response) Thanks for not having (i...,0


In [0]:
df = pd.DataFrame({'label':final_dataset.label, 'text':final_dataset.text})

In [0]:
# Split data into training and transfer
from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state = 12)

You can deactivate this warning by passing `no_check=True`.


/usr/local/lib/python3.6/dist-packages/fastai/basic_data.py:245: UserWarning: Your training dataloader is empty, you have only 28 items in your training set.
                 Your batch size is 64, you should lower it.
  Your batch size is {self.train_dl.batch_size}, you should lower it.""")


## UMLFit

**Steps**:

![alt text](https://www.novetta.com/wp-content/uploads/2019/02/ODSC-Blog-1024x214.png)


1. Load the data
2. Get a pre-trained language model
3. Train the language model
4. Train a classifier on top of the language model


## Language model

A language model learns about the domain. The data does not to be all labelled . What we are actually doing is learning about the domain with sequence generation techniques .

In [0]:
# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=28)

In [78]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.988268,3.505610,0.293429,00:09


In [79]:
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-4,1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,3.540807,3.342873,0.305162,00:12
1,3.371844,3.296992,0.318129,00:12
2,3.154292,3.293003,0.318757,00:12
3,2.923984,3.304364,0.321136,00:11
4,2.747485,3.312159,0.321777,00:11


In [84]:
learn.predict("I am anxious.", n_words=100)

'I am anxious. Present in your body and your body is n\'t sleeping my leg on the inside of our soul ? No , i think it \'s really not about having such a brain , you know , but there \'s anxiety and anxiety about the moment - like , " Hates yourself ? You say n\'t back , i \'m anyways . Less than a year . Yeah , that \'s of course . It \'s not healthy . RAMONA . i \'m not mad . " It \'s like all the'

In [0]:
learn.save('ft')
learn.save_encoder('ft_enc')

## Building a classifier

In [0]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5).to_fp16()
learn.load_encoder('ft_enc')

In [87]:
data_clas.show_batch()

text,target
"xxbos xxmaj how are you ? xxmaj um , similar i guess . i 'm just curious where you were when you called . xxmaj it was right after our session . xxmaj yeah . xxmaj like ten minutes later or something . xxmaj so i guess sometimes i 'm finding myself in these situations where i feel like completely overwhelmed . xxmaj and immediately like i don't - it",0
"xxbos xxmaj one link i could n’t click on for some reason , but i was having a problem , my computer had just xxunk , so i ’ll try it again . xxmaj can you send me another one i ’ll see if it works ? xxmaj okay . xxmaj as far as the letter xxunk , if you want me to put this address of the xxmaj english",0
"xxbos xxmaj where are you ? i guess okay . i guess i felt more depressed and anxious this past week , or for part of it . xxmaj in some context ? xxmaj ivan and i are having our scheduled conversations and i guess we ’ve talked a little bit more about the – i do n’t know if we talked about it , but it touched on the",1
"xxbos xxmaj claire , [ ph ] can you wait on that ? xxmaj like , i mean it ’s already the xxunk . xxmaj sure . i actually was going to ask you , because i have n’t xxunk , maybe , about , i think , four checks . xxmaj okay . xxmaj so i went to do it yesterday , and i realized i probably should ask",0
"xxbos xxmaj no , it 's not locked . xxmaj oh , my xxmaj god . xxmaj carl . xxmaj not locked . xxmaj yeah . xxmaj we 're good ? xxmaj yeah , we 're good . xxmaj awesome . xxmaj cool . i 've seen xxmaj ironman xxup iii three times since we last met , so that 's good . i 've seen two xxunk and one",0


In [88]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.656847,0.635527,0.850000,00:06


In [89]:
#Unfreezing a train a bit more
learn.unfreeze()
learn.fit_one_cycle(3, slice(1e-4, 1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.625847,0.631338,0.850000,00:06
1,0.574661,0.623798,0.850000,00:07
2,0.514494,0.600956,0.850000,00:07


In [90]:
learn.predict("This was a great movie!")

(Category 0, tensor(0), tensor([0.5864, 0.4136]))